In [ ]:
%matplotlib inline
from jupyterthemes import jtplot
jtplot.style(theme="onedork", context="notebook", ticks=True, grid=True)

import numpy as np
import h5py
import os
import sys
import glob

import matplotlib.pyplot as plt
import scipy.stats
import seaborn as sns
import pandas as pd
import tikzplotlib

# from tqdm import tqdm
from tqdm.notebook import tqdm_notebook as tqdm

import helper.circular
import fastpli.io
import fastpli.tools
import fastpli.objects
import fastpli.analysis

sim_path = "output/simulation/*.h5"
out_file = "output/analysis/"

In [ ]:
microscope = "PM"
model = "r"
df = pd.read_pickle(
    os.path.join(out_file,
                 f"cube_2pop_simulation_{microscope}_model_{model}_.pkl"))

df_acc = pd.read_pickle(
    os.path.join(
        out_file,
        f"cube_2pop_simulation_{microscope}_model_{model}_schilling.pkl"))

In [ ]:
data = df[(df.f0 == 0.0) & (df.f1 == 0.0)].explode("epa_dir")
grid = sns.FacetGrid(data,
                     col="omega",
                     hue="omega",
                     palette=sns.color_palette("dark", 10),
                     col_wrap=5,
                     height=2.5)
grid.map(plt.scatter, "psi", "epa_dir", marker="o")
grid.fig.tight_layout(w_pad=1)

In [ ]:
data = df[(df.f0 == 0.0) & (df.f1 == 0.0)].explode("epa_dir")

fig, axes = plt.subplots(2, 5)
for i, omega in enumerate(data.omega.unique()):
    sns.boxplot(x="psi",
                y="epa_dir",
                data=data[data.omega == omega],
                orient='v',
                color=sns.color_palette("dark", 10)[i],
                ax=axes[i // 5, i % 5])

data = df[(df.f0 == 0.0) & (df.f1 == 0.0)].explode("epa_ret")
fig, axes = plt.subplots(2, 5)
for i, omega in enumerate(data.omega.unique()):
    sns.boxplot(x="psi",
                y="epa_ret",
                data=data[data.omega == omega],
                orient='v',
                color=sns.color_palette("dark", 10)[i],
                ax=axes[i // 5, i % 5])

In [ ]:
f0_inc = 0
psi = 0.2
skip = 1

hist_bin = lambda n_phi: np.linspace(0, np.pi, n_phi + 1, endpoint=True)

sub = (df.f0_inc == f0_inc) & (df.psi == psi)

fig, axs = plt.subplots(len(df[sub].omega.unique()[::skip]),
                        len(df[sub].f1_rot.unique()[::skip]),
                        subplot_kw={'projection': 'polar'},
                        figsize=(20, 20))

axs = np.atleast_2d(axs)
for i in range(axs.shape[0]):
    for j in range(axs.shape[1]):
        axs[i, j].set_xticklabels([])
        axs[i, j].set_yticklabels([])

for i, f1_rot in enumerate(
        tqdm(np.sort(df[sub].f1_rot.unique())[::skip], leave=False)):
    for j, omega in enumerate(
            tqdm(np.sort(df[sub].omega.unique())[::skip], leave=False)):

        df_sub = df[sub & (df.f1_rot == f1_rot) & (df.omega == omega)]
        if df_sub.empty:
            continue

        # epa
        data = df_sub.explode("epa_dir").epa_dir.to_numpy(dtype=float)
        data = helper.circular.remap(data, np.pi, 0)
        h, x = np.histogram(data, hist_bin(72), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
        h = np.append(np.concatenate((h, h), axis=0), h[0])
        axs[j, i].plot(x, h, 'b')
        h_max = np.max(h)

        # rofl
        data = df_sub.explode("rofl_dir").rofl_dir.to_numpy(dtype=float)
        data = helper.circular.remap(data, np.pi, 0)
        h, x = np.histogram(data, hist_bin(72), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
        h = np.append(np.concatenate((h, h), axis=0), h[0])
        axs[j, i].plot(x, h, 'r')
        h_max = np.max(h)

        # ground truth
        df_o = pd.read_pickle(
            f"output/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
        )
        phi = df_o[(df_o.f0_inc == f0_inc)
                   & (df_o.f1_rot == f1_rot)].explode("phi").phi.to_numpy(float)

        phi = helper.circular.remap(phi, np.pi, 0)

        h, x = np.histogram(phi, hist_bin(180), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
        h = np.append(np.concatenate((h, h), axis=0), h[0])
        h = h / np.max(h) * h_max

        axs[j, i].plot(x, h, 'g')
        axs[j, i].set_title(f"o:{omega:.0f}, f1:{f1_rot:.1f}")

#         fig.savefig(f"phi_f0_{f0_inc}_f1_{f1_rot}.svg")

In [ ]:
f0_inc = 90
f1_rot = 0

# FIXME: f0_inc = 90, f1_rot = 90, were is omega=0?

skip = 1

dir_bin = lambda n: np.linspace(0, np.pi, n + 1, endpoint=True)
inc_bin = lambda n: np.linspace(-2*np.pi, 2*np.pi, n + 1, endpoint=True)

sub = (df.f0_inc == f0_inc) & (df.f1_rot == f1_rot)  #& (df.psi == psi)

fig, axs = plt.subplots(len(df[sub].psi.unique()[::skip]),
                        len(df[sub].omega.unique()[::skip]),
                        subplot_kw={'projection': 'polar'},
                        figsize=(20, 20))

axs = np.atleast_2d(axs)
for i in range(axs.shape[0]):
    for j in range(axs.shape[1]):
        axs[i, j].set_xticklabels([])
        axs[i, j].set_yticklabels([])
print(axs.shape)

for i, psi in enumerate(
        tqdm(np.sort(df[sub].psi.unique())[::skip], leave=False)):
    for j, omega in enumerate(
            tqdm(np.sort(df[sub].omega.unique())[::skip], leave=False)):

        df_sub = df[sub & (df.psi == psi) & (df.omega == omega)]
        if df_sub.empty:
            continue

        data = df_sub.explode("epa_dir").epa_dir.to_numpy(dtype=float)
        data = helper.circular.remap(data, np.pi, 0)
        h, x = np.histogram(data, dir_bin(72), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
        h = np.append(np.concatenate((h, h), axis=0), h[0])
        h = h / np.max(h)
        axs[i, j].plot(x, h, 'b')

        data = df_sub.explode("rofl_dir").rofl_dir.to_numpy(dtype=float)
        data = helper.circular.remap(data, np.pi, 0)
        h, x = np.histogram(data, dir_bin(72), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
        h = np.append(np.concatenate((h, h), axis=0), h[0])
        h = h / np.max(h)
        axs[i, j].plot(x, h, 'r')
        dir_rofl_max = np.max(h)
        
        data = df_sub.explode("rofl_inc").rofl_inc.to_numpy(dtype=float)
        h, x = np.histogram(data, inc_bin(72), density=True)
        h = h / np.max(h)
        x = x[:-1] + (x[1] - x[0]) / 2
        axs[i, j].plot(x, h, 'm')
        
        # ground truth
        df_o = pd.read_pickle(
            f"output/models/cube_2pop_psi_{psi:.1f}_omega_{omega:.1f}_.solved.pkl"
        )

        phi = df_o[(df_o.f0_inc == f0_inc)
                   & (df_o.f1_rot == f1_rot)].explode("phi").phi.to_numpy(float)
        phi = helper.circular.remap(phi, np.pi, 0)
        h, x = np.histogram(phi, dir_bin(180), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        x = np.append(np.concatenate((x, x + np.pi), axis=0), x[0])
        h = np.append(np.concatenate((h, h), axis=0), h[0])
        h = h / np.max(h)
        axs[i, j].plot(x, h, 'g')
        
        theta = df_o[(df_o.f0_inc == f0_inc)
                   & (df_o.f1_rot == f1_rot)].explode("theta").theta.to_numpy(float)
        alpha = np.pi/2 - theta
        h, x = np.histogram(alpha, inc_bin(180), density=True)
        x = x[:-1] + (x[1] - x[0]) / 2
        h = h / np.max(h)
        axs[i, j].plot(x, h, 'k')
        
        axs[i, j].set_title(f"o:{omega:.0f}, psi:{psi:.1f}")

#         fig.savefig(f"phi_f0_{f0_inc}_f1_{f1_rot}.svg")

In [ ]:
import helper.spherical_interpolation
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

f0_inc = 60
psi = 0.3

sub = (df_acc.psi == psi) & (df_acc.f0_inc == f0_inc)
f1_rot = df_acc[sub].f1_rot.to_numpy(float)
omega = df_acc[sub].omega.to_numpy(float)
data = df_acc[sub].acc.to_numpy(float)

print((data==0).any())

# get points on sphere
phi = []
theta = []
for f1, om in zip(f1_rot, omega):
    v = np.array([np.cos(np.deg2rad(om)), np.sin(np.deg2rad(om)), 0])
    rot_inc = fastpli.tools.rotation.y(-np.deg2rad(f0_inc))
    rot_phi = fastpli.tools.rotation.x(np.deg2rad(f1))
    rot = np.dot(rot_inc, rot_phi)
    v = np.dot(rot, v)
    theta.extend([np.arccos(v[2])])
    phi.extend([np.arctan2(v[1], v[0])])
    
phi_ = phi.copy()
theta_ = theta.copy()
data_ = data.copy()

# apply symmetries
phi = np.array(phi)
theta = np.array(theta)

phi = np.concatenate((phi, -phi), axis=0)
theta = np.concatenate((theta, theta), axis=0)
data = np.concatenate((data, data), axis=0)

phi = np.concatenate((phi, phi), axis=0)
theta = np.concatenate((theta, np.pi+theta), axis=0)
data = np.concatenate((data, data), axis=0)

# rm multiple
phi, theta = helper.spherical_interpolation.remap_sph_angles(phi, theta)
tmp = np.concatenate((np.atleast_2d(phi), np.atleast_2d(theta), np.atleast_2d(data)), axis=0)
tmp = np.unique(tmp, axis=1)
phi, theta, data = tmp[0,:], tmp[1,:], tmp[2,:]
    
# interplate mesh on sphere
x, y, z, data_i = helper.spherical_interpolation.on_mesh(
    phi, theta, data, 40, 40)

print(data_i.max())
print(data_i.min())
ax.plot_surface(x, y, z, facecolors=plt.cm.viridis(data_i))

x = np.multiply(np.cos(phi_), np.sin(theta_)) * 1.05
y = np.multiply(np.sin(phi_), np.sin(theta_)) * 1.05
z = np.cos(theta_) * 1.05

print(data_.max())
print(data_.min())
sc = ax.scatter(x, y, z, marker='o', s=50, c=data_, alpha=1, vmin = 0, vmax = 1, cmap="viridis")
plt.colorbar(sc)
ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
ax.view_init(30, 30)
plt.show()